In [1]:
%matplotlib ipympl

In [2]:
import requests as rq
import csv
import pandas as pd


In [ ]:
ny_times_timeseries_url = r'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

# John's Hopkins
# https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/03-31-2020.csv # regular format

In [16]:
def read_remote_csv(url):
    # read csv from url
    response = rq.get(url)
    csv_reader = csv.reader(response.text.split('\n'), delimiter=',')
    header = next(csv_reader)

    csv_data = [row for row in csv_reader]
    return (header, csv_data)

def read_local_csv(file_path):
    # read csv from url
    with open(file_path,'rt') as file:
        csv_reader = csv.reader(file, delimiter=',')
        header = next(csv_reader)

        csv_data = [row for row in csv_reader]
        
    return (header, csv_data)

In [5]:
header, csv_data = read_remote_csv(ny_times_timeseries_url)
case_data = pd.DataFrame.from_records(data=csv_data,columns=header)
case_data['date'] = pd.to_datetime(case_data['date'])
case_data[['cases','deaths']] = case_data[['cases','deaths']].applymap(int)

In [6]:
case_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [17]:
case_data.pivot_table(index='date', columns='state', values='cases', aggfunc=sum).plot.line(figsize=(15,15))

#.groupby(['date','state']).sum()['cases'].plot.line()


NameError: name 'case_data' is not defined

In [18]:
# cloned from https://github.com/CSSEGISandData/COVID-19

jh_path = r'../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-31-2020.csv'

In [20]:
header, csv_data = read_local_csv(jh_path)

In [21]:
header

['FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Last_Update',
 'Lat',
 'Long_',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active',
 'Combined_Key']

In [24]:
jh_data = pd.DataFrame.from_records(csv_data, columns=header)

In [25]:
jh_data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001,Abbeville,South Carolina,US,2020-03-31 23:43:56,34.22333378,-82.46170658,4,0,0,0,"Abbeville, South Carolina, US"
1,22001,Acadia,Louisiana,US,2020-03-31 23:43:56,30.295064899999996,-92.41419698,39,1,0,0,"Acadia, Louisiana, US"
2,51001,Accomack,Virginia,US,2020-03-31 23:43:56,37.76707161,-75.63234615,7,0,0,0,"Accomack, Virginia, US"
3,16001,Ada,Idaho,US,2020-03-31 23:43:56,43.4526575,-116.24155159999998,195,3,0,0,"Ada, Idaho, US"
4,19001,Adair,Iowa,US,2020-03-31 23:43:56,41.33075609,-94.47105874,1,0,0,0,"Adair, Iowa, US"
5,29001,Adair,Missouri,US,2020-03-31 23:43:56,40.19058551,-92.60078167,2,0,0,0,"Adair, Missouri, US"
6,40001,Adair,Oklahoma,US,2020-03-31 23:43:56,35.88494195,-94.65859267,4,0,0,0,"Adair, Oklahoma, US"
7,08001,Adams,Colorado,US,2020-03-31 23:43:56,39.87432092,-104.3362578,181,2,0,0,"Adams, Colorado, US"
8,17001,Adams,Illinois,US,2020-03-31 23:43:56,39.98815591,-91.18786813,2,0,0,0,"Adams, Illinois, US"
9,18001,Adams,Indiana,US,2020-03-31 23:43:56,40.7457653,-84.93671406,1,0,0,0,"Adams, Indiana, US"
